## TP_Projet

In [2]:
import re

data = open("CISI.ALLnettoye","r").readlines()
data=[i[:len(i)-1] for i in data] #Enleve le '\n'

collection = {}
titles=[]
for line in range(len(data)):
    if data[line][:3]==".I ":
        doc=[]
    elif data[line-1][:3]==".I ":
        titre="Document_"+str(data[line-1][3:])+": "+str(data[line])
        titles.append(titre)
    else:
        doc.append(data[line])
        collection[titre]=doc
len(collection)



1460

In [3]:
content = list(collection.values())
str(content[0]).lower()
dc=[[str(i).split() for i in content[l]]for l in range(len(content))]
len(dc)


1460

In [9]:
from nltk.tokenize import word_tokenize

#[word_tokenize(line) for line in content]

d=[[word_tokenize(line) for line in content[i]] for i in range(len(content))]
content

[['   The present study is a history of the DEWEY Decimal',
  'Classification.  The first edition of the DDC was published',
  'in 1876, the eighteenth edition in 1971, and future editions',
  "will continue to appear as needed.  In spite of the DDC's",
  'long and healthy life, however, its full story has never',
  'been told.  There have been biographies of Dewey',
  'that briefly describe his system, but this is the first',
  'attempt to provide a detailed history of the work that',
  'more than any other has spurred the growth of',
  'librarianship in this country and abroad.'],
 ['This report is an analysis of 6300 acts of use',
  'in 104 technical libraries in the United Kingdom.',
  'Library use is only one aspect of the wider pattern of',
  'information use.  Information transfer in libraries is',
  'restricted to the use of documents.  It takes no',
  'account of documents used outside the library, still',
  'less of information transferred orally from person',
  'to person.  

In [6]:
from nltk.corpus import stopwords
from tqdm import tqdm

documents_dict = dict()
for doc in tqdm(range(len(d))):
    word_occurence=dict()
    documents_dict[doc+1]=word_occurence
    for line in d[doc]:
        for word in line:
            wordl = word.lower()
            if len(wordl)>1 and wordl not in stopwords.words('english'):
                word_occurence[wordl]=0
for doc in tqdm(range(len(d))):
    for line in d[doc]:
        for word in line:
            wordl = word.lower()
            if len(wordl)>1 and wordl not in stopwords.words('english'):
                documents_dict[doc+1][wordl] += 1



100%|██████████| 1460/1460 [00:34<00:00, 42.35it/s]


In [7]:
documents_dict

{1: {'present': 1,
  'study': 1,
  'history': 2,
  'dewey': 2,
  'decimal': 1,
  'classification': 1,
  'first': 2,
  'edition': 2,
  'ddc': 2,
  'published': 1,
  '1876': 1,
  'eighteenth': 1,
  '1971': 1,
  'future': 1,
  'editions': 1,
  'continue': 1,
  'appear': 1,
  'needed': 1,
  'spite': 1,
  "'s": 1,
  'long': 1,
  'healthy': 1,
  'life': 1,
  'however': 1,
  'full': 1,
  'story': 1,
  'never': 1,
  'told': 1,
  'biographies': 1,
  'briefly': 1,
  'describe': 1,
  'system': 1,
  'attempt': 1,
  'provide': 1,
  'detailed': 1,
  'work': 1,
  'spurred': 1,
  'growth': 1,
  'librarianship': 1,
  'country': 1,
  'abroad': 1},
 2: {'report': 1,
  'analysis': 1,
  '6300': 1,
  'acts': 2,
  'use': 4,
  '104': 1,
  'technical': 3,
  'libraries': 3,
  'united': 1,
  'kingdom': 1,
  'library': 4,
  'one': 2,
  'aspect': 1,
  'wider': 1,
  'pattern': 1,
  'information': 7,
  'transfer': 2,
  'restricted': 1,
  'documents': 2,
  'takes': 1,
  'account': 1,
  'used': 1,
  'outside': 1,
  's

In [ ]:
def TF(doc,maxi):
    # maxi : Nb de docs analysés (maximum = N)
    tfD={}
    for i in tqdm(range(1,maxi)):
        tfDict = {}
        bagOfWordsCount = len(doc[i].keys())
        for word, count in doc[i].items():
            tfDict[word] = float("{:.15f}".format(count / float(bagOfWordsCount)))
        tfD[i] = tfDict
    return tfD

In [ ]:
import math
from tqdm import tqdm
def IDF(docu,maxi):
    # maxi : Nb de docs analysés (maximum = N)
    N = len(docu)
    sac_of_words={}
    words={}
    for i in tqdm(range(1,maxi)):
        wordDict={}
        for w in docu[i].keys():
            if w in sac_of_words.keys():
                wordDict[w] = sac_of_words[w]
            else:
                wordDict[w] = 0
                for doc in d[:maxi]:
                    isPresent=0
                    for line in doc:
                        for word in line:
                            if w==word.lower():
                                isPresent=1
                                wordDict[w] += 1
                            if isPresent:
                                break
                        if isPresent:
                            break
                wordDict[w] = math.log(maxi/wordDict[w])
                sac_of_words[w] = wordDict[w]
        words[i]=wordDict
    
    return words

In [ ]:
def TFIDF(tfD, idfD):
    maxi = len(tfD)
    if maxi != len(idfD):
        print("Pas le même nombre de doc")
    else:
        tfidf = {}
        for i in range(1,maxi):
            tfidf_inter = {}
            for word, val in tfD[i].items():
                tfidf_inter[word] = val * idfD[i][word]
            tfidf[i] = tfidf_inter
        return tfidf

In [ ]:
def sortDict(dict):
    dictionary={}
    for i in range(len(dict)):
        dico={}
        for k, v in sorted(dict[i].items(), key=lambda x: -x[1]):
            dico[k] = v
        dictionary[i] = dico
    return dictionary

In [ ]:
doc_max = len(documents_dict) # Nb de documents utilisés
tf = TF(documents_dict,doc_max)
idf = IDF(documents_dict,doc_max)

tfidf = TFIDF(tf,idf)
tfidf = sortDict(tfidf)
tfidf

 24%|██▍       | 354/1459 [01:46<05:33,  3.31it/s]


KeyboardInterrupt: 

In [ ]:
documents_dict

NameError: name 'documents_dict' is not defined

In [ ]:
IDF(documents_dict,50)

100%|██████████| 50/50 [00:01<00:00, 47.09it/s]


{0: {'the': 0.0,
  'present': 2.302585092994046,
  'study': 1.5141277326297755,
  'is': 0.38566248081198473,
  'a': 0.127833371509885,
  'history': 2.5257286443082556,
  'of': 0.0,
  'dewey': 3.2188758248682006,
  'decimal': 3.912023005428146,
  'classification': 2.5257286443082556,
  '.': 0.3566749439387324,
  'first': 2.120263536200091,
  'edition': 3.2188758248682006,
  'ddc': 3.912023005428146,
  'was': 1.0216512475319812,
  'published': 2.120263536200091,
  'in': 0.22314355131420976,
  '1876': 3.2188758248682006,
  ',': 0.127833371509885,
  'eighteenth': 3.912023005428146,
  '1971': 2.8134107167600364,
  'and': 0.10536051565782635,
  'future': 2.5257286443082556,
  'editions': 3.912023005428146,
  'will': 1.5141277326297755,
  'continue': 3.912023005428146,
  'to': 0.06187540371808745,
  'appear': 2.8134107167600364,
  'as': 0.5447271754416719,
  'needed': 3.912023005428146,
  'spite': 3.912023005428146,
  "'s": 1.3470736479666094,
  'long': 2.8134107167600364,
  'healthy': 3.9120

In [ ]:
tfidf[1438]

KeyError: 1438

In [ ]:
doc_max = 50 # Nb de documents utilisés
TF(documents_dict,doc_max)


{0: {'the': 0.136363636363636,
  'present': 0.015151515151515,
  'study': 0.015151515151515,
  'is': 0.03030303030303,
  'a': 0.03030303030303,
  'history': 0.03030303030303,
  'of': 0.090909090909091,
  'dewey': 0.03030303030303,
  'decimal': 0.015151515151515,
  'classification': 0.015151515151515,
  '.': 0.060606060606061,
  'first': 0.03030303030303,
  'edition': 0.03030303030303,
  'ddc': 0.03030303030303,
  'was': 0.015151515151515,
  'published': 0.015151515151515,
  'in': 0.060606060606061,
  '1876': 0.015151515151515,
  ',': 0.075757575757576,
  'eighteenth': 0.015151515151515,
  '1971': 0.015151515151515,
  'and': 0.045454545454545,
  'future': 0.015151515151515,
  'editions': 0.015151515151515,
  'will': 0.015151515151515,
  'continue': 0.015151515151515,
  'to': 0.03030303030303,
  'appear': 0.015151515151515,
  'as': 0.015151515151515,
  'needed': 0.015151515151515,
  'spite': 0.015151515151515,
  "'s": 0.015151515151515,
  'long': 0.015151515151515,
  'healthy': 0.0151515